# <br>




In [1]:
spark.version

3.1.2

In [2]:
import org.apache.commons.io.FileUtils
import org.apache.spark.sql.{DataFrame, Encoder, SparkSession}
import org.apache.spark.sql.functions.{col, explode, lit}
import org.apache.spark.sql.streaming.{StreamingQuery, Trigger}
import org.apache.spark.sql.types.StructType

In [3]:
import spark.implicits._
import org.apache.spark.sql._

In [4]:
import java.sql.Timestamp


case class DebeziumEvent(payload: Payload)
case class Payload(op: String, source: Source, before: String, after: String)
case class Source(ts_ms: Long, lsn: Long, table: String)
case class Member(id: Int,
                    c_d_id: Float,
                    first_name: String,
                    last_name: String,
                    address: String,
                    address_city: String,
                    address_country: String,
                    insurance_company: String,
                    insurance_number: String,
                    ts_created: String,
                    ts_updated: String)

case class AccidentClaims(claim_id: Int,
                  claim_total: Float,
                  claim_total_receipt: String,
                  claim_currency: String,
                  member_id: Int,
                  accident_date: String,
                  accident_type: String,
                  accident_detail: String,
                  claim_date: String,
                  claim_status: String,
                  ts_created: String,
                  ts_updated: String
                  )

In [5]:
val inputSchema = implicitly[Encoder[DebeziumEvent]].schema

In [6]:
val inputDF = spark
      .readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "pg_claims.claims.members")
      .option("startingOffsets", "earliest")
      .load()

In [7]:
 import org.apache.spark.sql._


val extractedDF = inputDF
      .selectExpr("CAST(value AS STRING)")
      .select(functions.from_json($"value", inputSchema).as("data"))

In [8]:
def writeAsDelta(deltaTableName: String, df: DataFrame) = {
    val query = df
      .writeStream
      .trigger(Trigger.ProcessingTime("5 seconds"))
      .foreachBatch { (batchDF: DataFrame, batchID: Long) =>
        println(s"Writing to Delta $batchID")
        batchDF.write
          .format("delta")
          .mode("append")
          .save(deltaTableName)
      }
      .outputMode("update")
      .start()
    query
  }

In [9]:
writeAsDelta("/tmp/data/staging/" + "members", extractedDF).awaitTermination()

org.apache.spark.sql.streaming.StreamingQueryException: Query [id = 45d0b91d-c2c1-4931-9e8a-4d09618311e8, runId = 60a88a40-09cf-45f6-a0fc-693a623a5f56] terminated with exception: org/apache/spark/kafka010/KafkaConfigUpdater

In [10]:
spark.read.format("delta").load("/tmp/data/staging/members").show()